In [1]:
print("hello world")

hello world


In [4]:
# Data ingestion
from langchain_community.document_loaders import TextLoader
loader=TextLoader("text_data.txt")
text_document = loader.load()
text_document

[Document(metadata={'source': 'text_data.txt'}, page_content='The Taj Mahal (/ˌtɑːdʒ məˈhɑːl, ˌtɑːʒ -/ TAHJ mə-HAHL, TAHZH -\u2060; Hindustani: [taːdʒ ˈmɛɦ(ɛ)l]; lit.\u2009\'Crown of the Palace\') is an ivory-white marble mausoleum on the right bank of the river Yamuna in Agra, Uttar Pradesh, India. It was commissioned in 1631 by the fifth Mughal emperor, Shah Jahan (r.\u20091628–1658) to house the tomb of his beloved wife, Mumtaz Mahal; it also houses the tomb of Shah Jahan himself. The tomb is the centrepiece of a 17-hectare (42-acre) complex, which includes a mosque and a guest house, and is set in formal gardens bounded on three sides by a crenellated wall.\n\nConstruction of the mausoleum was completed in 1648, but work continued on other phases of the project for another five years. The first ceremony held at the mausoleum was an observance by Shah Jahan, on 6 February 1643, of the 12th anniversary of the death of Mumtaz Mahal. The Taj Mahal complex is believed to have been compl

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [11]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

## load, chunnk and index the content of the html page
loader=WebBaseLoader(web_path=("https://whc.unesco.org/en/list/252/"),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=( "rich-text", "tab_content", "tab_content_en")
                         )))

web_document=loader.load()

In [12]:
web_document

[Document(metadata={'source': 'https://whc.unesco.org/en/list/252/'}, page_content="An immense mausoleum of white marble, built in Agra between 1631 and 1648 by order of the Mughal emperor Shah Jahan in memory of his favourite wife, the Taj Mahal is the jewel of Muslim art in India and one of the universally admired masterpieces of the world's heritage.Immense mausolée funéraire de marbre blanc édifiée entre 1631 et 1648 à Agra sur l'ordre de l'empereur moghol Shah Jahan pour perpétuer le souvenir de son épouse favorite, le Taj Mahal, joyau le plus parfait de l'art musulman en Inde, est l'un des chefs-d'œuvre universellement admirés du patrimoine de l'humanité.إنّ تاج محل هو ضريح جنائزي هائل شُيّد من الرخام الأبيض بين عامي 1631 و1648 في أغرا بناءً على أوامر الإمبراطور المغولي شاه جهان بهدف تخليد ذكرى زوجته المفضّلة. ويشكّل تاج محل الذي يُعتبر أفضل جوهرة في الفن الإسلامي في الهند إحدى أبرز تُحف التراث البشري التي هي محطّ إعجاب العالم بأسره.泰姬陵是一座由白色大理石建成的巨大陵墓清真寺，是莫卧儿皇帝沙贾汗(Shah Jahan)为纪念

In [27]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("TajMahal.pdf")
pdf_document=loader.load()
#pdf_text = [doc.page_content for doc in pdf_document]

In [28]:
from langchain.schema import Document
# Step 2: Extract only the text content from each page
page_texts = [doc.page_content for doc in pdf_document]

# Step 3: Convert page texts back into Document objects (needed by LangChain splitter)
documents = [Document(page_content=text) for text in page_texts]

In [29]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,)
process_pdf = text_splitter.split_documents(documents)
process_pdf[:10]

[Document(metadata={}, page_content='See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/360018181\nTaj Mahal\nArticle · April 2022\nDOI: 10.1007/978-3-319-51726-1_3510-1\nCITATIONS\n0\nREADS\n3,889\n1 author:\nAmita Sinha\nIndian Institute of Technology BHU\n112 PUBLICATIONS\xa0\xa0\xa0430 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nAll content following this page was uploaded by Amita Sinha on 18 April 2022.\nThe user has requested enhancement of the downloaded file.'),
 Document(metadata={}, page_content='T\nTaj Mahal\nAmita Sinha\nIndependent Scholar, Urbana, IL, USA\nIntroduction\nThe Taj’s astonishing journey from memorial to\nmonument in the last 375 years is the story of a\nremarkable building becoming a national icon and\nof many ways of seeing it. The Taj represented\nimperial Mughal power at its zenith in the Indian\nsubcontinent and was intended to be a world\nfamous masterpiece of human creation. Built by\nthe Mughal Empero

In [32]:
## Vector embeddings
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="llama2",
)


db=FAISS.from_documents(documents=process_pdf,
                       embedding=embeddings)

In [33]:
db

In [34]:
query="Are there any UNESCO World Heritage Sites in India?"
docs=db.similarity_search(query=query, k=5)
docs[0].page_content

'left their marks on the sandstoneﬂooring, both\nsymbols and names, indicating that over\n400 teams were employed at the site (Koch\n2006). The Taj is built of brick and faced with\nred sandstone, white marble, and polished lime\nplaster. The red sandstone from Fatehpur Sikri\n(40 km from Agra) and white marble from\nMakrana in Rajasthan (about 400 km away)\nwere hauled on carts drawn by bullocks or buffa-\nloes. The building materials were hoisted over\nmud ramps, some said to be close to 2 miles in\ntheir length. Twenty-eight types of precious and\nsemi-precious stones inlaid into the white marble\nincluded agate from Yemen, turquoise from Tibet,\nlapis lazuli from Afghanistan, amethyst from Per-\nsia, sapphire from Ceylon, carnelian from Arabia,\noynx from Deccan, and garnets from Bundel-\nkhand (Baig and Mehrotra2017). Polished plaster\n(chuna), less expensive than marble, was used in\nupper storey ambulatory rooms and their domes,\ngiving them a white shining surface (Koch2006).'

In [35]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

C:\Users\shrey\AppData\Local\Temp\ipykernel_9944\2814759709.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama2")


In [43]:
from langchain_core.prompts import ChatPromptTemplate
prompt =ChatPromptTemplate.from_template("""
    Answer the following question based on the context below.
    Think stept by step and provide a final answer.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.   
    <context>
    {context}
    </context>
    Question: {input}""")

In [44]:
## Chai Introduction
## Create stuff document

from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt=prompt)

A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Retrievers can be created from vector stores, but are also broad enough to include Wikipedia search and Amazon Kendra.

In [45]:
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020DDB1891D0>, search_kwargs={})

In [46]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [48]:
response = retrieval_chain.invoke({"input": "When was the Taj Mahal built and who built it and why?"})

In [49]:
response['answer']

"Based on the context provided, the Taj Mahal was built in the 17th century, specifically between 1632 and 1653, during the reign of Mughal Emperor Shah Jahan. The Taj Mahal was built as a mausoleum for Shah Jahan's wife, Mumtaz Mahal, who died during childbirth in 1631. The construction of the Taj Mahal was overseen by the emperor himself and was completed after several years of work by thousands of workers, including architects, engineers, and artisans.\n\nThe Taj Mahal was built as a symbol of Shah Jahan's love and devotion to his wife, as well as his desire to create a monument that would stand the test of time. The complex includes the mausoleum itself, as well as several other structures, including gardens, pools, and walkways. The Taj Mahal is considered one of the greatest examples of Mughal architecture and is widely regarded as one of the most beautiful buildings in the world."

In [50]:
print(response['answer'])

Based on the context provided, the Taj Mahal was built in the 17th century, specifically between 1632 and 1653, during the reign of Mughal Emperor Shah Jahan. The Taj Mahal was built as a mausoleum for Shah Jahan's wife, Mumtaz Mahal, who died during childbirth in 1631. The construction of the Taj Mahal was overseen by the emperor himself and was completed after several years of work by thousands of workers, including architects, engineers, and artisans.

The Taj Mahal was built as a symbol of Shah Jahan's love and devotion to his wife, as well as his desire to create a monument that would stand the test of time. The complex includes the mausoleum itself, as well as several other structures, including gardens, pools, and walkways. The Taj Mahal is considered one of the greatest examples of Mughal architecture and is widely regarded as one of the most beautiful buildings in the world.
